# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4918, uptime 0:00:17
mini_beamline                    RUNNING   pid 4919, uptime 0:00:17
random_walk                      RUNNING   pid 4920, uptime 0:00:17
random_walk_horiz                RUNNING   pid 4921, uptime 0:00:17
random_walk_vert                 RUNNING   pid 4922, uptime 0:00:17
simple                           RUNNING   pid 4923, uptime 0:00:17
thermo_sim                       RUNNING   pid 4924, uptime 0:00:17
trigger_with_pc                  RUNNING   pid 4925, uptime 0:00:17


In [2]:
%run scripts/beamline_configuration.py

## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 1     Time: 2020-05-15 20:58:24
Persistent Unique Scan ID: '9723b4b7-4804-4a1e-8a79-17192038912b'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+
|         1 | 20:58:24.2 |     -1.000 |      98487 |


|         2 | 20:58:24.3 |     -0.500 |     102215 |
|         3 | 20:58:24.3 |      0.000 |     102532 |
|         4 | 20:58:24.3 |      0.500 |     102239 |
|         5 | 20:58:24.3 |      1.000 |     100334 |


+-----------+------------+------------+------------+
generator scan ['9723b4b7'] (scan num: 1)





('9723b4b7-4804-4a1e-8a79-17192038912b',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/9723b4b7-4804-4a1e-8a79-17192038912b-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 2     Time: 2020-05-15 20:58:24
Persistent Unique Scan ID: 'f859d896-bb53-4dc9-a8a2-7e51be6758f3'


New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 20:58:24.8 |          -1 | 1126001560 |


|         2 | 20:58:24.9 |          -0 | 1128858312 |


|         3 | 20:58:25.0 |           0 | 1131343627 |


|         4 | 20:58:25.1 |           0 | 1131760783 |


|         5 | 20:58:25.2 |           1 | 1130822883 |


+-----------+------------+-------------+------------+
generator scan ['f859d896'] (scan num: 2)





('f859d896-bb53-4dc9-a8a2-7e51be6758f3',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 3     Time: 2020-05-15 20:58:25
Persistent Unique Scan ID: '5c2d580b-b510-42b8-9743-f84b87d6c8b1'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 20:58:25.7 | 1117506789 |


|         2 | 20:58:26.6 | 1060807679 |


|         3 | 20:58:27.6 | 1069578597 |


...

|        28 | 20:58:52.7 | 1126425217 |


|        29 | 20:58:53.7 | 1117252068 |


|        30 | 20:58:54.7 | 1059219831 |


+-----------+------------+------------+
generator count ['5c2d580b'] (scan num: 3)





('5c2d580b-b510-42b8-9743-f84b87d6c8b1',)